In [1]:
import keypress_recognition.models as models
from keypress_recognition import dataset_p as dataset
from keypress_recognition import dataset_t
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext autoreload
%autoreload 2

In [2]:
concat = False
delta = 3 if concat else 1
channels = 3 * delta

In [3]:
mod = {}
mod['black'] = models.get_black_bundle_model(channels)
mod['black'].load_model('keypress_recognition/models/best_black_bundle.tar')
mod['white'] = models.get_white_bundle_model(channels)
mod['white'].load_model('keypress_recognition/models/best_white_bundle.tar') 

Trying to load model from keypress_recognition/models/best_black_bundle.tar ...
done
Trying to load model from keypress_recognition/models/best_white_bundle.tar ...
done


In [4]:
dataset.load_all_data(spliter=['test'], color=['black', 'white'], size=['bundle'], keypress=True, concatenate=concat, delta=delta)

Start seperating keyboard ...
  White single width: 21px
  Black single width: 20px
  White bundle width: 37px
  Black bundle width: 36px


IntProgress(value=0, max=5000)

In testset: 
  # of pressed black key: 761
  # of unpressed black key: 22409
  # of pressed white key: 10831
  # of unpressed white key: 22882


In [5]:
def visualize_keys(note):
    """
    note: bool
    return: str
    """
    return ("Y" if note else "N")

def display_concat_img(img, idx=None):
    if concat:
        if idx is None:
            return img[:, :, :, 3:6]
        else:
            return img[idx, :, :, 3:6]
    if idx is None:
        return img
    else:
        return img[idx, :, :, :]


In [6]:
for color in ('black', 'white'):
    print(color)
    results = mod[color].test(dataset, size='bundle', color=color, concatenate=concat)
    for k, v in results.items():
        print("{:10}: {:.4f}".format(k.title(), v))

black


IntProgress(value=0, max=44818)

Total data count 44864 Batch size 64
Precision : 0.9877
Recall    : 0.9947
Accuracy  : 0.9912
F1 Score  : 0.9912
white


IntProgress(value=0, max=45764)

Total data count 45824 Batch size 64
Precision : 0.9567
Recall    : 0.9705
Accuracy  : 0.9633
F1 Score  : 0.9635


In [31]:
# ! we disable threshold at first
blacknpy = np.empty((0, 36))
whitenpy = np.empty((0, 52))
for whites, blacks in dataset_t.data_batch(size='bundle'):
    blackout = mod['black'].evaluate(blacks, threshold=None).cpu().numpy()
    whiteout = mod['white'].evaluate(whites, threshold=None).cpu().numpy()
#     out = dataset_t.merge_two_colors(whiteout, blackout)
    blacknpy = np.concatenate((blacknpy, blackout[None]), axis=0)
    whitenpy = np.concatenate((whitenpy, whiteout[None]), axis=0)
print(blacknpy.dtype, blacknpy.shape)

IntProgress(value=0, max=5000)

float64 (5000, 36)


In [ ]:
label = np.load('keypress_recognition/dataset/y_test/y_test.npy')
label = label[:, 21:109] > 0
print(label.shape)

black_mask = np.array([1, 4, 6, 9, 11, 13, 16, 18, 21, 23, 25, 28, 30, 33, 35, 37, 40, 42, 45, 47, 49, 52, 54, 57, 59, 61, 64,
    66, 69, 71, 73, 76, 78, 81, 83, 85])
white_mask = np.array([0, 2, 3, 5, 7, 8, 10, 12, 14, 15, 17, 19, 20, 22, 24, 26, 27, 29, 31, 32, 34, 36, 38, 39, 41, 43, 44, 46, 48,
    50, 51, 53, 55, 56, 58, 60, 62, 63, 65, 67, 68, 70, 72, 74, 75, 77, 79, 80, 82, 84, 86, 87])

In [40]:
blackpred = blacknpy > 0.5
print(dataset_t.evaluate(blackpred, label[:, black_mask]))
whitepred = whitenpy > 0.5
print(dataset_t.evaluate(whitepred, label[:, white_mask]))

{'accuracy': 0.9856944444444444, 'precision': 0.010787486515641856, 'recall': 0.026281208935611037, 'F1 score': 0.015296367112810709}
{'accuracy': 0.8991269230769231, 'precision': 0.10278637770897833, 'recall': 0.18391653586926415, 'F1 score': 0.1318724967726987}


In [30]:
pred = dataset_t.merge_two_colors(whitepred, blackpred)
print(pred.shape)
eva = dataset_t.evaluate(pred, label)
print(eva)

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]]


In [11]:
np.save('keypress_recognition/dataset/y_pred/y_pred_bundle.npy', npy)

(5000, 88)
